In [ ]:
!pip install pandas

In [ ]:
!pip install gcsfs

In [ ]:
import pandas as pd


In [ ]:
columns = ['order_id', 'order_date', 'order_customer_id', 'order_status']

In [ ]:
!gsutil ls gs://abhiretail/pythondemo/retail_db/orders

In [ ]:
df = pd.read_csv('gs://abhiretail/pythondemo/retail_db/orders/part-00000' , 
                 names = columns)

In [ ]:
df.count()

In [12]:
!pip install pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.0/26.0 MB 42.8 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [13]:
import glob, os , pandas as pd

In [21]:
items = glob.glob(f'{src_file_path}/**' , recursive=True)
items

['retail_db/',
 'retail_db/customers',
 'retail_db/customers/part-00000',
 'retail_db/products',
 'retail_db/products/part-00000',
 'retail_db/create_db_tables_pg.sql',
 'retail_db/departments',
 'retail_db/departments/part-00000',
 'retail_db/README.md',
 'retail_db/order_items',
 'retail_db/order_items/part-00000',
 'retail_db/create_db.sql',
 'retail_db/schemas.json',
 'retail_db/orders',
 'retail_db/orders/part-00000',
 'retail_db/categories',
 'retail_db/categories/part-00000',
 'retail_db/load_db_tables_pg.sql']

In [25]:
files = list(filter(lambda item:os.path.isfile(item) and item.endswith('part-00000') , items))
files

['retail_db/customers/part-00000',
 'retail_db/products/part-00000',
 'retail_db/departments/part-00000',
 'retail_db/order_items/part-00000',
 'retail_db/orders/part-00000',
 'retail_db/categories/part-00000']

In [26]:
def get_file_names(src_file_path):
    items = glob.glob(f'{src_file_path}/**' , recursive=True)
    files = list(filter(lambda item: os.path.isfile(item) and item.endswith('part-00000') , items))
    return files

In [33]:
import json

def get_col_names(schema_file, ds_name):
    schemas = json.load(open(schema_file))
    ds_schema = sorted(schemas[ds_name] , key = lambda col: col['column_position'])
    return [col['column_name'] for col in ds_schema]

In [34]:
get_col_names('retail_db/schemas.json' , 'orders')

['order_id', 'order_date', 'order_customer_id', 'order_status']

In [36]:
for file in files:
    print(file)

retail_db/customers/part-00000
retail_db/products/part-00000
retail_db/departments/part-00000
retail_db/order_items/part-00000
retail_db/orders/part-00000
retail_db/categories/part-00000


In [39]:
src_file_path = 'retail_db'
files = get_file_names(src_file_path)
tgt_file_path = 'retail_db_parquet'
bucket = 'abhiretail'
schemas_file = 'retail_db/schemas.json'
for file in files:
    print(f'Uploading file {file}')
    blob_suffix = '/'.join(file.split('/')[1:])
    ds_name = file.split('/')[-2]
    blob_name = f'gs://{bucket}/{tgt_file_path}/{blob_suffix}.snappy.parquet'
    columns = get_col_names(schemas_file, ds_name)
    df = pd.read_csv(file , names = columns)
    df.to_parquet(blob_name , index=False)



Uploading file retail_db/customers/part-00000
Uploading file retail_db/products/part-00000
Uploading file retail_db/departments/part-00000
Uploading file retail_db/order_items/part-00000
Uploading file retail_db/orders/part-00000
Uploading file retail_db/categories/part-00000


In [43]:
!gsutil ls gs://abhiretail/retail_db_parquet/orders/part-00000.snappy.parquet

gs://abhiretail/retail_db_parquet/orders/part-00000.snappy.parquet


In [45]:
df = pd.read_parquet('gs://abhiretail/retail_db_parquet/orders/part-00000.snappy.parquet')